In [2]:
using Random
A = rand(0:99999, 100000)

100000-element Vector{Int64}:
 82123
 47877
 76995
 39660
 65069
 73189
 43664
 93801
 23180
 17377
     ⋮
 90008
 78703
 90516
 76820
  9844
 42400
 54824
 98934
 37856

In [6]:
function sort(A::AbstractVector)
    return sort!(copy(A))
end

function sort!(A::AbstractVector)
    @inbounds for i in 2:length(A)
        @inbounds key = A[i]
        j = i - 1
        while j >= 1 && A[j] > key
            j -= 1
        end
        @inbounds @views A[j+2:i] = A[j+1:i-1]
        @inbounds A[j+1] = key
    end
    return A
end

function sortperm!(A::AbstractVector)
    @views I = collect(1:length(A))
    @inbounds for i in 2:length(A)
        @inbounds key = A[i]
        @inbounds j = i - 1
        # сравниваем элементы вставляемого элемента и элемента на позиции j, 
        # пока не найдем место для вставки
        while j >= 1 && A[j] > key
            j -= 1
        end
        # вставляем элемент key в нужную позицию, 
        # сдвигая все элементы на позиции j+1, ..., i-1 на одну позицию вправо
        @views @inbounds A[j+2:i] = A[j+1:i-1]
        @inbounds A[j+1] = key
        # При каждой перестановке элементов в массиве A
        # необходимо произвести соответствующую перестановку
        # в массиве индексов I, чтобы сохранить соответствие между ними.
        @views @inbounds I[j+2:i] = I[j+1:i-1]
        @inbounds I[j+1] = i
    end
    return I
end

function sortperm(A::AbstractVector)
    return sortperm!(copy(A))
end


@showtime sort(A)
@showtime sortperm(A)
@showtime Base.sort(A; alg=InsertionSort)
@showtime Base.sortperm(A; alg=InsertionSort)


sort(A): 10.777934 seconds (216.05 k allocations: 18.605 GiB, 7.92% gc time, 0.50% compilation time)


sortperm(A): 14.631082 seconds (444.59 k allocations: 37.210 GiB, 5.72% gc time, 0.35% compilation time)


Base.sort(A; alg = InsertionSort): 1.644164 seconds (63.19 k allocations: 4.088 MiB, 3.67% compilation time)


Base.sortperm(A; alg = InsertionSort): 6.404495 seconds (51.74 k allocations: 3.407 MiB, 0.16% gc time, 1.36% compilation time)


100000-element Vector{Int64}:
 51217
 73956
 16852
 30424
 86690
 72472
  9896
 73092
 76885
 77327
     ⋮
 33318
 59550
 22103
 12536
 77491
  7173
 42901
 52819
 43093

In [4]:
function bubblesort!(a)
    n = length(a)
    for i in 1:n-1
        for j in 1:n-i
            if a[j] > a[j+1]
                a[j], a[j+1] = a[j+1], a[j]
            end
        end
    end
    return a
end

function bubblesort(A::AbstractVector)
    return bubblesort!(copy(A))
end

function comb_sort!(a; factor=1.2473309)
    step = length(a)
    while step >= 1
        for i in 1:length(a)-step
            if a[i] > a[i+step]
                a[i], a[i+step] = a[i+step], a[i]
            end
        end
        step = Int(floor(step/factor))
    end
    # Теперь массив почти упорядочен, осталось сделать всего несколько итераций внешнего цикла в bubble_sort!(a)
    bubblesort!(a)
end

function comb_sort(A::AbstractVector)
    return comb_sort!(copy(A))
end

@showtime bubblesort(A)
@showtime comb_sort(A)

bubblesort(A): 20.898476 seconds (18.81 k allocations: 1.762 MiB, 0.16% compilation time)


comb_sort(A): 6.215902 seconds (18.97 k allocations: 1.775 MiB, 0.65% compilation time)


100000-element Vector{Int64}:
     0
     1
     4
     7
     7
    10
    11
    11
    12
    13
     ⋮
 99987
 99988
 99989
 99989
 99991
 99991
 99991
 99995
 99997

In [5]:
function shell_sort!(
    a; 
    step_series = (length(a)÷2^i for i in 1:Int(floor(log2(length(a))))) 
)
    for step in step_series
        for i in firstindex(a):lastindex(a)-step
            j = i
            while j >= firstindex(a) && a[j] > a[j+step]
                a[j], a[j+step] = a[j+step], a[j]
                j -= step
            end
        end
    end
    return a
end

function shell_sort(A::AbstractVector)
    return shell_sort!(copy(A))
end

@showtime shell_sort(A)

shell_sort(A): 0.243177 seconds (41.49 k allocations: 2.945 MiB, 77.26% compilation time)


100000-element Vector{Int64}:
     0
     1
     4
     7
     7
    10
    11
    11
    12
    13
     ⋮
 99987
 99988
 99989
 99989
 99991
 99991
 99991
 99995
 99997

In [13]:
@inline function Base.merge!(a1, a2, a3)::Nothing # @inline - делает функцию "встраиваемой", т.е. во время компиляции ее тело будет встроено непосредственно в код вызывающей функции (за счет этого происходит экономия на времени, затрачиваемым на вызов функции; это время очень небольшое, но тем не менее)
    i1, i2, i3 = 1, 1, 1
    @inbounds while i1 <= length(a1) && i2 <= length(a2) # @inbounds - передотвращает проверки выхода за пределы массивов
        if a1[i1] < a2[i2]
            a3[i3] = a1[i1]
            i1 += 1
        else
            a3[i3] = a2[i2]
            i2 += 1
        end
        i3 += 1
    end
    @inbounds if i1 > length(a1)
        a3[i3:end] .= @view(a2[i2:end]) # Если бы тут было: a3[i3:end] = @view(a2[i2:end]), то это привело бы к лишним аллокациям (к созданию промежуточного массива)
    else
        a3[i3:end] .= @view(a1[i1:end])
    end
    nothing
end


function merge_sort!(a)
    b = similar(a) # - вспомогательный массив того же размера и типа, что и массив a
    N = length(a)
    n = 1 # n - текущая длина блоков
    @inbounds while n < N
        K = div(N,2n) # - число имеющихся пар блоков длины n
        for k in 0:K-1
            merge!(@view(a[(1:n).+k*2n]), @view(a[(n+1:2n).+k*2n]), @view(b[(1:2n).+k*2n]))
        end
        if N - K*2n > n # - осталось еще смержить блок длины n и более короткий остаток
            merge!(@view(a[(1:n).+K*2n]), @view(a[K*2n+n+1:end]), @view(b[K*2n+1:end]))
        elseif 0 < N - K*2n <= n # - оставшуюся короткую часть мержить не с чем
            b[K*2n+1:end] .= @view(a[K*2n+1:end])
        end
        a, b = b, a
        n *= 2
    end
    if isodd(log2(n)) # - если цикл был выполнен нечетное число раз, то b - это исходная ссылка на массив (на внешний массив), и a - это ссылка на вспомогательный массив (локальный)
        b .= a # b = copy(a) - это было бы не то же самое, т.к. при этом получилась бы ссылка на новый массив, который создает функция copy
        a = b
    end
    return a # - исходная ссылка на внешний массив (проверить, что это так, можно с помощью ===)
end


function merge_sort(A::AbstractVector)
    return merge_sort!(copy(A))
end

@showtime merge_sort(A)
@showtime Base.sort(A; alg=MergeSort)

merge_sort(A): 0.357206 seconds (602.62 k allocations: 27.011 MiB, 2.50% gc time, 96.08% compilation time)
Base.sort(A; alg = MergeSort): 0.008867 seconds (4 allocations: 1.145 MiB)


100000-element Vector{Int64}:
     1
     3
     6
     7
     7
     8
    10
    10
    10
    13
     ⋮
 99992
 99992
 99993
 99994
 99995
 99996
 99996
 99996
 99998

In [16]:
function part_sort!(A, b)
    N = length(A)
    K=0
    L=0
    M=N
    #ИНВАРИАНТ: A[1:K] < b && A[K+1:L] == b && A[M+1:N] > b
    while L < M 
        if A[L+1] == b
            L += 1
        elseif A[L+1] > b
            A[L+1], A[M] = A[M], A[L+1]
            M -= 1
        else # if A[L+1] < b
            L += 1; K += 1
            A[L], A[K] = A[K], A[L]
        end
    end
    return K, M+1 
    # 1:K и M+1:N - эти диапазоны индексов определяют ещё не 
    # отсортированные части массива A
end

function quick_sort!(A)
    if isempty(A)
        return A
    end
    N = length(A)
    K, M = part_sort!(A, A[rand(1:N)]) # - "базовый" элемент массива выбирается случайнам образом
    quick_sort!(@view A[1:K])
    quick_sort!(@view A[M:N])
    return A
end

function quick_sort(A::AbstractVector)
    return quick_sort!(copy(A))
end

@showtime quick_sort(A)
@showtime Base.sort(A; alg=QuickSort)

quick_sort(A): 0.119954 seconds (358.36 k allocations: 17.787 MiB, 89.69% compilation time)
Base.sort(A; alg = QuickSort): 0.005783 seconds (2 allocations: 781.297 KiB)


100000-element Vector{Int64}:
     1
     3
     6
     7
     7
     8
    10
    10
    10
    13
     ⋮
 99992
 99992
 99993
 99994
 99995
 99996
 99996
 99996
 99998

In [26]:
function order_statistics!(A::AbstractVector{T}, i::Integer)::T where T
    function part_sort!(indexes_range::AbstractUnitRange, b)
        K, L, M = indexes_range[1]-1, indexes_range[begin]-1, indexes_range[end] # 0, 0, N
        #ИНВАРИАНТ: A[indexes_range[begin]:K] < b && A[K+1:L] == b && A[M+1:indexes_range[end]] > b
        while L < M 
            if A[L+1] == b
                L += 1
            elseif A[L+1] > b
                A[L+1], A[M] = A[M], A[L+1]
                M -= 1
            else # if A[L+1] < b
                L += 1; K += 1
                A[L], A[K] = A[K], A[L]
            end
        end    
        return indexes_range[begin]:K, M+1:indexes_range[end] 
        # - эти диапазоны индексов определяют ещё не отсортированные части массива A
    end

    function find(indexes_range)
        left_range, right_range = part_sort!(indexes_range, A[rand(indexes_range)]) 
        # - здесь "базовый" элемент массива выбирается случайным образом
        if i in left_range
            return find(left_range) 
        elseif i in right_range
            return find(right_range)
        else
            return A[i]
        end
    end

    find(firstindex(A):lastindex(A))
end

order_statistics(A, i) = order_statistics!(copy(A), i)


@showtime order_statistics(A, 50000)

order_statistics(A, 50000): 0.074327 seconds (36.38 k allocations: 2.566 MiB, 97.48% compilation time)


49883

In [33]:
function calc_sort!(A)
    min_val, max_val = extrema(A)
    num_val = zeros(Int, max_val-min_val+1) # - число всех возможных значений
    for val in A
        num_val[val-min_val+1] += 1
    end  
    k = 0
    for (i, num) in enumerate(num_val)
        A[k+1:k+num] .= min_val+i-1
        k += num
    end
    return A
end

calc_sort(A) = calc_sort!(copy(A))


@showtime calc_sort(A)

calc_sort(A): 0.013571 seconds (775 allocations: 1.569 MiB, 64.64% compilation time)


100000-element Vector{Int64}:
     0
     1
     4
     7
     7
    10
    11
    11
    12
    13
     ⋮
 99987
 99988
 99989
 99989
 99991
 99991
 99991
 99995
 99997